In [35]:
#DONNÉES 
#=============================================================
"""set of alphabet"""
alpha = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','-']

"""Read a file with name and mode, return a list of lines"""
def read_file(filename, mode):
    f = open(filename,mode);
    lines = f.readlines();
    f.close();
    return lines;

"""Transform these lines of format FASTA into a list of seqs without sympol"""
def get_seq(lines):
    list_seqs = []
    for line in lines:
        if(line[0] == '>'):
            print("It's a comment about the seq's name");
        else:
            list_seqs.append(line.split());
    return list_seqs;

#def scanner(filename,)

In [38]:
#TEST
#=============================================================

#print(len(alpha))
#lines_D = read_file("Dtrain.txt",'r');
#for line in lines_D:
#    print(line);
#lines_test1 = read_file("test_seq.txt",'r');
#for line in lines_test1:
#    print(line)
#print(lines_test1[1])
#lines_test1[0][0]
#len(lines_test1[1].split()[0]) # to confirm the seq'length without symbol
#get_seq(lines_D)
